In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import sklearn as sk
import tensorflow_addons as tfa
import cv2 as cv

# data_preprocessing.py

In [2]:
def preprocess(ds, batch_size, ds_type='train'):
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    num_classes = 3

    resize_and_rescale = create_resize_and_rescale_layer()
    data_augmentation = create_augmentation_layer()

    if ds_type == 'test':
        ds = ds.map(lambda ds: (ds['image'], ds['label']))

    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE) # disable to visualise train/ valid iamges
    ds = ds.batch(batch_size)

    if ds_type == 'train':
        ds = ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE)
    
    if ds_type == 'train' or ds_type == 'valid': 
        ds = ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
    
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


def create_resize_and_rescale_layer(): 
    resize_and_rescale = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE[0], IMG_SIZE[1], interpolation='bilinear'),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
    ], name='resize_and_rescale')
    return resize_and_rescale


def create_augmentation_layer():
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        # tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1),
        # RandomInvert(),
        # RandomSheer(),
        # RandomGaussionFilter(),
    ], name='data_augmentation')
    return data_augmentation


def make_generator(seed=None):
    if seed:
        return tf.random.Generator.from_seed(seed)
    else:
        return tf.random.Generator.from_non_deterministic_state()


def random_invert_img(x, p=0.5):
    if tf.random.uniform([]) < p:
        x = (255-x)
    else:
        x
    return x


class RandomInvert(tf.keras.layers.Layer):
    def __init__(self, factor=0.5, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor
    
    def call(self, x):
        return random_invert_img(x)



class RandomSheer(tf.keras.layers.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)
        self.rng = make_generator(seed)


    def call(self, x):
        x_level = self.rng.uniform(shape=[], minval=0.1, maxval=0.35)
        y_level = self.rng.uniform(shape=[], minval=0.1, maxval=0.35)

        x = tfa.image.shear_x(x, x_level, 0)
        x = tfa.image.shear_y(x, y_level, 0)
        return x


class RandomGaussionFilter(tf.keras.layers.Layer):
    def __init__(self, seed=None, **kwargs):
        super().__init__(**kwargs)


    def call(self, x):
        k = np.random.randint(3, 11)
        sigma = np.random.uniform(low=0.1, high=0.9)
        x = tfa.image.gaussian_filter2d(x, filter_shape=(k, k), sigma=sigma)
        return x


def plot(acc, val_acc, loss, val_loss, initial_epochs=0):
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1.0])
    plt.title('Training and Validation Accuracy')
    if initial_epochs != 0:
        plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Fine Tuning')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    if initial_epochs != 0:
        plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Fine Tuning')
    plt.show()


def save_results(save_path, image_ids, predicted_labels):
    results = image_ids.drop('image', axis=1)
    results.columns = ['ID', 'Label']
    results['Label'] = predicted_labels 
    results = results.sort_values('ID').reset_index(drop=True)
    results.to_csv(RESULT_SAVE_PATH, index=False)

# model.py

In [3]:
def create_base_model(img_shape):
    # base_model = tf.keras.applications.MobileNetV2(
    #     input_shape=img_shape,
    #     include_top=False,
    #     weights='imagenet'
    # )
    base_model = tf.keras.applications.Xception(
        input_shape=img_shape,
        include_top=False,
        weights='imagenet',
    )
    # base_model = tf.keras.applications.ResNet152V2(
    #     input_shape=img_shape,
    #     include_top=False,
    #     weights='imagenet',
    # )
    base_model.trainable = False
    return base_model


class AdaptiveConcatPooling(tf.keras.layers.Layer):
    def __init__(self):
        super(AdaptiveConcatPooling, self).__init__()

    
    def call(self, x):
        output_size = (x.shape[1], x.shape[2])
        avg_pool = tfa.layers.AdaptiveAveragePooling2D(output_size)(x)
        max_pool = tfa.layers.AdaptiveMaxPooling2D(output_size)(x)
        return tf.concat([avg_pool, max_pool], axis=1)


def create_prediction_layer(num_classes):
    # Float 32
    # prediction = tf.keras.Sequential([
    #     tf.keras.layers.GlobalAveragePooling2D(),
    #     tf.keras.layers.Dropout(0.5),
    #     tf.keras.layers.Dense(num_classes, activation='softmax'),
    # ], name='prediction')

    # Mixed precision 
    prediction = tf.keras.Sequential([
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, name='logits'),
        tf.keras.layers.Activation('softmax', dtype='float32', name='probs'),
    ], name='prediction')
    
    # Adaptive
    # prediction = tf.keras.Sequential([
    #     AdaptiveConcatPooling(),
    #     tf.keras.layers.Flatten(),
    #     tf.keras.layers.BatchNormalization(),
    #     tf.keras.layers.Dropout(0.75),
    #     tf.keras.layers.Dense(num_classes, name='logits'),
    #     tf.keras.layers.Activation('softmax', dtype='float32', name='probs'),
    # ], name='prediction')
    return prediction


def create_model(image_shape, num_classes):
    base_model = create_base_model(image_shape)
    prediction_layer = create_prediction_layer(num_classes)

    inputs = tf.keras.Input(shape=image_shape)
    x = base_model(inputs, training=False)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    return model


def create_fine_tune_model(model, fine_tune_at=100):
    print("Number of layers in the base model: ", len(model.layers[1].layers))
    model.layers[1].trainable = True
    # Freeze all the layers before the `fine_tune_at` layer
    for layer in model.layers[1].layers[:fine_tune_at]:
        layer.trainable = False
    
    # Unfreeze the top layers while leaving BatchNorm layers frozen
    # for layer in model.layers[1].layers[-fine_tune_at:]: 
    #     if not isinstance(layer, tf.keras.layers.BatchNormalization):
    #         layer.trainable = True
    #         print(layer)

    return model

# train.py

In [4]:
def compile_and_fit(model, train_ds, valid_ds, hyperparams, start_epoch=0, name=''):
    if start_epoch != 0:
        total_epochs =  hyperparams['initial_epochs'] + hyperparams['fine_epochs']
        lr = hyperparams['fine_learning_rate']
    else:
        total_epochs =  hyperparams['initial_epochs']
        lr = hyperparams['learning_rate']

    log_dir = os.path.join('logs', name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=hyperparams['label_smoothing']),
        metrics=['accuracy']
    )
    print(model.summary())
    history = model.fit(
        train_ds,
        epochs=total_epochs,
        initial_epoch=start_epoch,
        validation_data=valid_ds,
        callbacks=[tensorboard_callback],
        # verbose=0,
    )
    return model, history

# main.py

In [5]:
# pre pre-process images
# python preprocess.py

# Build dataset
# python -m tensorflow_datasets.scripts.download_and_prepare --datasets=mri_dataset --module_import=datasets.mri_dataset --manual_dir=data/processed --data_dir=data/

## GPU Setup

In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable first GPU
  tf.config.set_visible_devices(physical_devices[1:], 'GPU')
  logical_devices = tf.config.list_logical_devices('GPU')
  # Logical device was not created for first GPU
  assert len(logical_devices) == len(physical_devices) - 1
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

## Enable Mixed Precision

In [7]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

Some of your GPUs may run slowly with dtype policy mixed_float16 because they do not all have compute capability of at least 7.0. Your GPUs:
  Tesla V100S-PCIE-32GB, compute capability 7.0
  Tesla V100-PCIE-32GB, compute capability 7.0
  Tesla P100-PCIE-16GB, compute capability 6.0 (x2)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


## Load Dataset

In [8]:
import tensorflow_datasets as tfds
from datasets.mri_dataset import MriDataset
from tensorflow.data import Dataset
from tensorflow.keras.callbacks import TensorBoard


data_folder = 'data'
raw_folder = 'raw'
processed_folder = 'processed'
dataset = 'mri_dataset'
train_folder = 'train'
test_folder = 'test'
train_label = 'train_label.csv'

RESULT_SAVE_PATH = 'submission.csv'
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
TEST_BATCH_SIZE = 64
IMG_SIZE = (512, 512)
IMG_SHAPE = IMG_SIZE + (3,)
SEED = 0
AUTOTUNE = tf.data.experimental.AUTOTUNE


tf.random.set_seed(SEED)
num_classes = 3

train_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
valid_folds = tfds.load(
    name='mri_dataset', 
    split=[f'train[{k}%:{k+10}%]' for k in range(0, 100, 20)],
    download=False, 
    shuffle_files=False, 
    as_supervised=True,
    data_dir=data_folder
)
test_ds_raw, test_info_raw = tfds.load(
    name='mri_dataset', 
    split='test', 
    download=False, 
    shuffle_files=False, 
    as_supervised=False, 
    with_info=True,
    data_dir=data_folder
)

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_folds = [ preprocess(ds, batch_size=TRAIN_BATCH_SIZE, ds_type='train') for ds in train_folds ]
valid_folds = [ preprocess(ds, batch_size=VALID_BATCH_SIZE, ds_type='valid') for ds in valid_folds ]
test_ds = preprocess(test_ds_raw, batch_size=TEST_BATCH_SIZE, ds_type='test')

In [9]:
train_valid_df = pd.read_csv(os.path.join(data_folder, processed_folder, train_label))
train_ds = train_folds[0]
valid_ds = valid_folds[0]

print(f'Number of train batches: {train_ds.cardinality()}')
print(f'Number of valid batches: {valid_ds.cardinality()}')
print(f'Number of test batches: {test_ds.cardinality()}')

Number of train batches: 17
Number of valid batches: 2
Number of test batches: 5


## Visualise Dataset

In [10]:
# # disable image resize and rescale in preprocess function ONLY for visualisation
# # Train Data
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(9):
#     for i in range(9):
#       ax = plt.subplot(3, 3, i + 1)
#       plt.imshow(images[i].numpy().astype("uint8"))
#       plt.title(labels[i].numpy())
#       plt.axis("off")

In [11]:
# # Test Data
# plt.figure(figsize=(10, 10))
# for i, ds in enumerate(test_ds_raw.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(ds['image'].numpy().astype("uint8"))
#     plt.title('ID: {}'.format(ds['id'].numpy()))
#     plt.axis("off")

In [12]:
# TODO: Fix visualisation
# data_augmentation = create_augmentation_layer()

# for image, _ in train_ds.take(1):
#   plt.figure(figsize=(10, 10))
#   first_image = image[0]
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
#     plt.imshow(augmented_image[0] / 255)
#     plt.axis('off')

## Train and Validate

In [13]:
%%time

tf.random.set_seed(SEED)
hyperparams = {
    'initial_epochs': 150,
    'fine_epochs': 50,
    'learning_rate': 1e-4,
    'fine_learning_rate': 1e-5,
    'label_smoothing': 0.1,
    'fine_tune_at': 10,
}

train_accs, valid_accs, train_losses, valid_losses = [], [], [], []
train_folds = [train_folds[3]] + train_folds[0:3] + [train_folds[4]] # re-order folds because fold-4 hardest to train
valid_folds = [valid_folds[3]] + valid_folds[0:3] + [valid_folds[4]]
for i, (train_ds, valid_ds) in enumerate(zip(train_folds, valid_folds)):
    k = i + 1
    experiment_name = f'inception-{k}_fold'
    print(f'# -------------------- {k} fold -------------------- #')
    tf.keras.backend.clear_session()
    model = create_model(IMG_SHAPE, num_classes)
    
    print(f'{k} fold pre train:') 
    model, history = compile_and_fit(model, train_ds, valid_ds, hyperparams, name=experiment_name)
    print()

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    plot(acc, val_acc, loss, val_loss)
    print()

    print(f'{k} fold fine train:')
    model = create_fine_tune_model(model, fine_tune_at=hyperparams['fine_tune_at'])
    model, history_fine = compile_and_fit(model, train_ds, valid_ds, hyperparams, start_epoch=history.epoch[-1], name=experiment_name)
    print()

    acc += history_fine.history['accuracy']
    val_acc += history_fine.history['val_accuracy']
    loss += history_fine.history['loss']
    val_loss += history_fine.history['val_loss']
    plot(acc, val_acc, loss, val_loss, initial_epochs=hyperparams['initial_epochs']+1)
    print()

    # record accuracy
    train_acc = history.history['accuracy'][-1]
    valid_acc = history.history['val_accuracy'][-1]
    train_loss = history.history['loss'][-1]
    valid_loss = history.history['val_loss'][-1]

    train_accs.append(train_acc)
    valid_accs.append(valid_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    print(f'{k} fold | Train Loss: {train_loss} | Train Accuracy: {train_acc} | Validation Loss: {valid_loss} | Validation Accuracy: {valid_acc}\n')

# average accuracy
avg_train_acc = np.mean(train_accs)
avg_valid_acc = np.mean(valid_accs)
avg_train_loss = np.mean(train_loss)
avg_valid_loss = np.mean(valid_loss)

print(f'Avg Train Loss: {avg_train_loss} | Avg Train Accuracy: {avg_train_acc} | Avg Validation Loss: {avg_valid_loss} | Avg Validation Accuracy: {avg_valid_acc}')

# -------------------- 1 fold -------------------- #
1 fold pre train:
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 16, 16, 2048)      20861480  
_________________________________________________________________
prediction (Sequential)      (None, 3)                 6147      
Total params: 20,867,627
Trainable params: 6,147
Non-trainable params: 20,861,480
_________________________________________________________________
None
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


KeyboardInterrupt: 

## Predict

In [14]:
tf.random.set_seed(SEED)

# TODO: combine ds and retrain with all data
# combined_ds = train_folds[0].concatenate(valid_folds[0])

In [15]:
# TODO: analyse predicted results
# image_batch, label_batch = valid_ds.as_numpy_iterator().next()
# predictions = model.predict_on_batch(image_batch)
# predicted_indices = tf.argmax(predictions, 1)
# predicted_labels = predicted_indices.numpy()


# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   plt.title(f'pred: {predicted_labels[i]} true: {label_batch[i]}')
#   plt.axis("off")

## Evaluate

In [16]:
# predict test labels
predictions = model.predict(test_ds)
predicted_indices = tf.argmax(predictions, 1)
predicted_labels = predicted_indices.numpy()
predicted_labels

array([2, 1, 1, 0, 2, 0, 2, 1, 0, 2, 2, 1, 1, 0, 1, 2, 0, 0, 1, 1, 2, 1,
       1, 2, 1, 0, 2, 2, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       2, 1, 2, 0, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 2, 0, 0, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 0, 1, 2, 1, 1, 0, 2, 1,
       1, 0, 1, 2, 1, 2, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 1,
       1, 0, 2, 1, 1, 2, 1, 1, 0, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 0, 1, 1,
       1, 2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 0, 1, 2, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 2, 0, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 0, 2, 2, 0, 1, 0, 2, 0, 0, 0,
       2, 1, 1, 0, 1, 0, 2, 2, 1, 0, 2, 0, 1, 1, 2, 2, 2, 1, 2, 0, 1, 2,
       0, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1, 2, 2, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       2, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 1, 1, 0, 0, 2,
       0, 2, 0, 2, 1, 1, 1, 0, 1, 2, 0, 1, 0, 0, 1, 1, 2, 2, 1, 2, 0, 0,
       0, 2, 0, 2, 1, 1])

In [17]:
# save results
img_ids = tfds.as_dataframe(test_ds_raw, test_info_raw)
save_results(RESULT_SAVE_PATH, img_ids, predicted_labels)